# Presist Results of Snowpark Dataframe

1. Presist the Results of snowpark DataFrames

    Create views from DataFrames

    Save DataFrame results as Snowflake Tables

    Save DataFrame results as files in stage

2. Enhance perfomance in Snowpark applications

    Materialized results(caching)

    caching DataFrame(using.cache_results) and understand why this is useful
    
    create a temprary table

For more information follows the below links:

1. [snowflake.snowpark.DataFrameWriter](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrameWriter#snowflake.snowpark.DataFrameWriter)

2.[snowflake.snowpark.DataFrame.cache_result](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.cache_result)

3. [HOW TO: COPY Data from CSV Files INTO Snowflake Table using Snowpark?](https://thinketl.com/how-to-copy-data-from-csv-files-into-snowflake-table-using-snowpark/)

4. [Using Persisted Query Results](https://docs.snowflake.com/en/user-guide/querying-persisted-results)

In [ ]:
#Creating Session:
from snowflake.snowpark.functions import col
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
#Creating Tables

df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER"
    ).filter(col("C_NATIONKEY") == '15'
    ).select(col("C_CUSTKEY"), col("C_NAME"))
df

In [ ]:
df.schema
df.schema.names

In [ ]:
df.write.mode("overwrite").save_as_table("snowpark_db.sourced.customer")
session.table("snowpark_db.sourced.customer").count()

In [ ]:
#Append: Will add data 
df.write.mode("append").save_as_table("snowpark_db.sourced.customer")
session.table("snowpark_db.sourced.customer").count()

In [ ]:
#Ignore will create duplicates
df.write.mode("ignore").save_as_table("snowpark_db.sourced.customer")
session.table("snowpark_db.sourced.customer").count()

In [ ]:
#ErrorIfExists: Will throw error on 
df.write.mode("errorifexists").save_as_table("snowpark_db.sourced.customer")
session.table("snowpark_db.sourced.customer").count()

In [ ]:
df.write.mode("errorifexists").save_as_table("snowpark_db.sourced.customer_t", table_type="temp")
session.table("snowpark_db.sourced.customer_t").count()

In [ ]:
#Will create Views
view = df.create_or_replace_view("snowpark_db.works.customer_vw")
session.table("snowpark_db.works.customer_vw").count()

In [ ]:
#Write to Location
df.write.copy_into_location(
    "@snowpark_db.sourced.stage1/customer.parquet",
    file_format_type="parquet",
    header=True, overwrite=True, single=True)

In [ ]:
list @snowpark_db.sourced.stage1

In [ ]:
#Reader method
df2 = session.read.parquet("@snowpark_db.sourced.stage1/customer.parquet")
df2

In [ ]:
# recreate table
session.sql("drop table if exists snowpark_db.sourced.customer_f").collect()

# COPY INTO table (from dataframe)
df2.copy_into_table("snowpark_db.sourced.customer_f", force=True)

# load table data
df = session.table("snowpark_db.sourced.customer_f")
df

In [ ]:
#Create and Populate a temp table(From cache_results())

session.sql("create or replace temp table cache_t(num int)").collect()
session.sql("insert into cache_t values (1), (2)").collect()
df = session.table("cache_t")
df

In [ ]:
# cache df into a temp table
df_cached = df.cache_result()
df_cached.is_cached
df_cached

In [ ]:
# the cached result does not change!
session.sql("insert into cache_t values (3)").collect()
df
df_cached

In [ ]:
# clear cached result
df_cached.drop_table()
df_cached